In [4]:
import numpy as np 
import matplotlib.pyplot as plt 
import csv
import math
cm = 1/2.54
textwidth = 13.98611*cm

%matplotlib qt
normalsize = 12
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Cambria",
    "font.size": normalsize
})


In [119]:
# 1D arrays 
def flow(Ch,q,title,xaxis,yaxis,filename):
    W = 300.0
    H = 300.0
    x = np.arange(0,W+0.1,0.1) 
    y = np.arange(-H,H+0.1,0.1) 
    y[-1] = H

    # Meshgrid 
    X,Y = np.meshgrid(x,y)
    print
    
    # Assign vector directions
    Fa = -abs(Ch/X) - abs(0.515*(Y-q) / (1+abs(Y-q)) - 0.513)
    # Fa = -abs(Ch/X) - abs(-1 / (1 + np.exp(Y-q)))
    Fb = Y/X - Fa*X
    Ftheta = np.arctan(2*Fa*X + Fb)

    Ex = -np.cos(Ftheta)
    Ey = -np.sin(Ftheta)

    start_x = np.arange(0,H+10,10)

    stream_points = np.column_stack((start_x, np.sqrt(H*H - start_x*start_x)))
    stream_points = np.append(stream_points, stream_points*[1,-1], axis=0)

    # Depict illustration
    fig = plt.figure()
    plt.streamplot(X,Y,Ex,Ey, start_points=stream_points, density=5, linewidth=None, color='black')
    plt.axhline(0,color='black',linestyle=':')
    plt.gca().set_aspect('equal')

    fig.set_figheight(15*cm)
    # plt.title(title, pad=10)
    plt.xlabel(xaxis, fontsize=normalsize-1)
    plt.ylabel(yaxis, fontsize=normalsize-1)
    plt.show
    plt.savefig(filename, format="pdf", bbox_inches="tight")
flow(2,11,'End Effector Movement Path','Horizontal Distance From Target (mm)','Vertical Distance From Target (mm)',"foot_path.pdf")
# flow(0,0,r'End Effector Movement Path',r'Horizontal Distance From Target (mm)',r'Vertical Distance From Target (mm)',"foot_path.pdf")


/tmp/ipykernel_15187/2060055378.py:14: RuntimeWarning: divide by zero encountered in divide
  Fa = -abs(Ch/X) - abs(0.515*(Y-q) / (1+abs(Y-q)) - 0.513)
/tmp/ipykernel_15187/2060055378.py:16: RuntimeWarning: divide by zero encountered in divide
  Fb = Y/X - Fa*X
/tmp/ipykernel_15187/2060055378.py:16: RuntimeWarning: invalid value encountered in multiply
  Fb = Y/X - Fa*X
/tmp/ipykernel_15187/2060055378.py:17: RuntimeWarning: invalid value encountered in multiply
  Ftheta = np.arctan(2*Fa*X + Fb)


In [116]:
points1 = np.array([(0, 0), (50, 10), (100, -25)])
points2 = np.array([(0, 0), (25, 45), (50, 10)])
# get x and y vectors
x = points1[:,0]
y = points1[:,1]
x2 = points2[:,0]
y2 = points2[:,1]

# calculate polynomial
z = np.polyfit(x, y, 3)
f = np.poly1d(z)
z2 = np.polyfit(x2, y2, 3)
f2 = np.poly1d(z2)


# calculate new x's and y's
x_wanted = np.linspace(x[0], x[1], 50)
y_wanted = f(x_wanted)
x_real2 = np.linspace(x[1], x[2], 50)
y_real2 = f(x_real2)
x_real1 = np.linspace(x2[0], x2[2], 50)
y_real1 = f2(x_real1)

fig = plt.figure()
plt.plot(x_wanted, y_wanted,'--', color='black')
plt.plot(x_real1, y_real1, color='black')
plt.plot(x_real2, y_real2, color='black')
plt.plot(x[0],y[0],'o', color='black')
plt.plot(x[2],y[2],'o', color='black')
plt.plot(x[1],y[1],'s', markersize=7, color='black')
plt.axhline(0,color='black',linestyle=':')
plt.axvline(0,color='black',linestyle=':')
plt.text(x[1]+2, y[1]+2,'Recomputation point')
plt.legend(['Desired Arc', 'Generated Arc'])
plt.gca().set_aspect('equal')

fig.set_figwidth(textwidth)
# plt.title(r'Problem With Arc Recomputation', pad=10)
plt.xlabel(r'Horizontal Position (mm)', fontsize=normalsize-1)
plt.ylabel(r'Vertical Position (mm)', fontsize=normalsize-1)

# plt.show()
plt.savefig("old_path.pdf", format="pdf", bbox_inches="tight")


/home/epep/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/epep/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


In [115]:
x = np.linspace(-50,50,1000)
Ex = x
def Ey(q):
    return 0.515*(x-q)/(1+abs(x-q))-0.505
fig = plt.figure()
plt.plot(Ex,Ey(-20),'--', color='black')
plt.plot(Ex,Ey(0), '-',color='black')
plt.plot(Ex,Ey(20), '-.', color='black')
plt.axhline(0,linestyle=':',color='black')
plt.axvline(0,linestyle=':',color='black')
plt.legend([r'\(q=-20\)', r'\(q=-0\)', r'\(q=20\)'])

# plt.gca().set_aspect('equal')
# plt.gca().set_ylim([-1,1])

fig.set_size_inches(textwidth,textwidth*0.45)
# plt.title(r'Sigmoid Like \(S(y)\)', pad=10)
plt.xlabel(r'Vertical Separation (mm)', fontsize=normalsize-1)
# plt.ylabel(r'Vertical Position (mm)', fontsize=normalsize-1)

plt.savefig("sigmoid_like.pdf", format="pdf", bbox_inches="tight")

In [176]:
def euler_from_quaternion(x, y, z, w):
        """
        Convert a quaternion into euler angles (roll, pitch, yaw)
        roll is rotation around x in radians (counterclockwise)
        pitch is rotation around y in radians (counterclockwise)
        yaw is rotation around z in radians (counterclockwise)
        """
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        roll_x = math.atan2(t0, t1)
     
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch_y = math.asin(t2)
     
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        yaw_z = math.atan2(t3, t4)
     
        return roll_x, pitch_y, yaw_z # in radians

x = []
y = []

roll = []
pitch = []
height_set = []
height_true = []
with open('/home/epep/Documents/HexapodMasters/body.csv','r') as csvfile: 
    plots = csv.reader(csvfile, delimiter = ',') 
    for row in plots:
        x.append(round(float(row[0])*15,2))
        y.append(round(float(row[2])*15,2))
        r,p,_ = euler_from_quaternion(float(row[3]), float(row[4]),float(row[5]),float(row[6]))
        roll.append(np.rad2deg(r))
        pitch.append(np.rad2deg(p))
        height_set.append(round(float(row[7])*15,2))
        height_true.append(round(float(row[8])*15,2))
start = 50
end = -1
fig,ax = plt.subplots(3)
bheight, = ax[0].plot(x[start:end], y[start:end], color='black', label='Body Height')
dbheight, = ax[0].plot(x[start:end], height_set[start:end],'--', color='black', label='Desired Body Height')
theight, = ax[0].plot(x[start:end], height_true[start:end],':', color='black', label='Terrain Height')
ax[1].plot(x[start:end], roll[start:end], ':', color='black', label='Roll')
ax[1].plot(x[start:end], pitch[start:end], '--', color='black', label='Pitch')

# ax[0].set(title=r'Body Height and Rotation Against Forward Distance', ylabel=r'Height (cm)')
ax[0].set(ylabel=r'Height (cm)')
ax[1].set(ylabel=r'Rotation (Degrees)')
# ax[0].legend(bbox_to_anchor=(1, 0.95),loc='upper right')
# ax[0].legend()

# first_legend = ax[0].legend(handles=[bheight], loc='upper left')
# second_legend = ax[0].legend(handles=[dbheight],bbox_to_anchor=(0, 0.15), loc='lower left')
# ax[0].add_artist(first_legend)
# ax[0].add_artist(second_legend)
# ax[0].legend(handles=[theight], loc='lower right')

first_legend = ax[0].legend(handles=[bheight, theight], loc='lower left')
ax[0].add_artist(first_legend)
ax[0].legend(handles=[dbheight],bbox_to_anchor=(1, 0.5), loc='lower right')

ax[1].legend()
# ax[0].margins(0.01,0.01)
# ax[1].margins(0.01,0.01)

leg_i = 2
x = []
y = []
height_true = []
with open('/home/epep/Documents/HexapodMasters/feet.csv','r') as csvfile:
    plots = csv.reader(csvfile, delimiter = ',') 
    for row in plots:
        x.append(round(float(row[0+leg_i*3])*15,2))
        y.append(round(float(row[2+leg_i*3])*15,2))
        height_true.append(round(float(row[18+leg_i])*15,2))

ax[2].plot(x[start:end],y[start:end],color='black', label='Foot Height')
ax[2].plot(x[start:end],height_true[start:end],'--',color='black', label='Terrain Height')
ax[2].legend(bbox_to_anchor=(0.5, 0.95))
ax[2].set(xlabel =r'X Position (cm)')
ax[2].set(ylabel=r'Height (cm)')
plt.gca().set_aspect('equal')

fig.set_size_inches(textwidth,textwidth*1.4)
plt.show()
        
plt.savefig("data_org.pdf", format="pdf", bbox_inches="tight")
